In [34]:
import os
import pickle
import logging

import numpy as np
import pandas as pd
import click
from typing import Union
from sklearn.metrics import precision_score
import sklearn

from msap.modeling.model_evaluation.statistics import (
    get_embedded_data,
    get_selected_features,
    get_curve_metrics,
    get_curve_metrics_test,
    get_training_statistics,
    get_baseline_training_statistics,
    get_validation_statistics,
    get_baseline_validation_statistics,
    get_testing_statistics,
    get_baseline_testing_statistics,
    get_similarity_matrix)
from msap.explanatory_analysis import get_pairwise_correlation
from msap.utils import (
    ClassifierHandler,
    load_X_and_y,
    KFold_by_feature)
from msap.utils.plot import (
    plot_heatmap,
    plot_embedded_scatter,
    plot_rfe_line,
    plot_rfe_line_detailed,
    plot_curves,
    plot_confusion_matrix)
from msap.modeling.configs import (
    ModelSelectionConfig)

In [35]:
METHODS_PC = ['pearson', 'spearman', 'kendall']
METHODS_EMBEDDING = ['tsne', 'pca']
METHODS_CURVE = ['pr', 'roc']
CLASSIFIER_MODES = [
    'decisiontreeclassifier',
    'gaussiannb',
    'multinomialnb',
    'svc',
    'adaboostclassifier',
    'randomforestclassifier',
    'mlpclassifier']

In [36]:
def plot_all_confusion_matrices(
        clf: sklearn.base.BaseEstimator,
        X_train: pd.DataFrame,
        y_train: pd.Series,
        X_test: pd.DataFrame,
        y_test: pd.Series,
        path_output_dir: str,
        use_smote_first: bool,
        use_rfe: bool,
        use_f1: bool,
        splits: Union[int, list[list, list]] = None,
        classifier: str = None):
    # for baseline, predict all positive/depressed
    if use_smote_first and use_rfe:
        fileprefix = "cm_smote_rfe"
    elif use_smote_first and not use_rfe:
        fileprefix = "cm_smote"
    elif not use_smote_first and use_rfe:
        fileprefix = "cm_rfe"
    else:
        fileprefix = "cm"
    if classifier is not None:
        fileprefix = "_".join([classifier, fileprefix])

    if use_f1:
        mode = "f1"
    else:
        mode = "balanced_accuracy"

    best_cv_result = get_validation_statistics(clf, X_train, y_train, splits)
    plot_confusion_matrix(
        cv_result=best_cv_result,
        axis_labels=['Depressed', 'Not Depressed'],
        mode=mode,
        path_save=f"{path_output_dir}/{fileprefix}_val.svg")

    best_cv_result_val_baseline = get_baseline_validation_statistics(
        clf, X_train, y_train, splits)
    plot_confusion_matrix(
        cv_result=best_cv_result_val_baseline,
        axis_labels=['Depressed', 'Not Depressed'],
        mode=mode,
        path_save=f"{path_output_dir}/{fileprefix}_val_baseline.svg")

    # Plot confusion matrix with various metrics for training.
    best_cv_result_train = get_training_statistics(
        clf, X_train, y_train, splits)
    plot_confusion_matrix(
        cv_result=best_cv_result_train,
        axis_labels=['Depressed', 'Not Depressed'],
        mode=mode,
        path_save=f"{path_output_dir}/{fileprefix}_train.svg")

    best_cv_result_train_baseline = get_baseline_training_statistics(
        clf, X_train, y_train, splits)
    plot_confusion_matrix(
        cv_result=best_cv_result_train_baseline,
        axis_labels=['Depressed', 'Not Depressed'],
        mode=mode,
        path_save=f"{path_output_dir}/{fileprefix}_train_baseline.svg")

    # Plot confusion matrix with various metrics for testing.
    best_cv_result_test = get_testing_statistics(
        clf, X_train, y_train, X_test, y_test, splits)
    plot_confusion_matrix(
        cv_result=best_cv_result_test,
        axis_labels=['Depressed', 'Not Depressed'],
        mode=mode,
        path_save=f"{path_output_dir}/{fileprefix}_test.svg")

    # Plot confusion matrix with various metrics for baseline testing.
    cv_result_test_baseline = get_baseline_testing_statistics(
        clf, X_train, y_train, X_test, y_test, splits)
    plot_confusion_matrix(
        cv_result=cv_result_test_baseline,
        axis_labels=['Depressed', 'Not Depressed'],
        mode=mode,
        path_save=f"{path_output_dir}/{fileprefix}_baseline_test.svg")

In [37]:
def parse_model_selection_result(ms_result: tuple, mode: str) -> list:
    """Parse the model selection result tuple and get the best models.

    Args:
        ms_result: Model selection result tuple.

    Returns:
        List of best model and statistics for each classifiers.

    """
    candidates, _ = ms_result
    candidates = [(i, c, cv['best_f1']) for i, c, cv in candidates]

    if mode == 'f1':
        f1s_mean = []
        for i, c, cv_best in candidates:
            # Iterate over splits to calculate average F1 score.
            f1s = [cv_best[f'split_{j}']['f1']
                   for j in range(int(len(cv_best)/2))]
            f1s_mean += [np.mean(np.nan_to_num(f1s))]

        candidates = list(zip(candidates, f1s_mean))
        candidates = sorted(candidates, key=lambda x: x[1], reverse=True)

        best_candidate_per_clf = []
        for clf in CLASSIFIER_MODES:
            for (i, c, cv_best), f1_mean in candidates:
                if c[3] == clf:
                    if cv_best['param'] is not None:
                        cv_best['param'] = {k.split('__')[-1]: v
                                            for k, v in cv_best['param'].items()}

                    best_candidate_per_clf += [((i, c, cv_best), f1_mean)]
                    break
        return best_candidate_per_clf
    elif mode == 'balanced_accuracy':
        candidates, _ = ms_result
        # candidates = [(i, c, cv) for i, c, cv in candidates]
        balanced_accuracys_mean = []
        grid_results = []
        for i, c, cv in candidates:
            # parse every grid search result
            for key in cv:
                # Iterate over splits to calculate average F1 score for clf
                result = cv[key]
                balanced_accuracys = [
                    result[f'split_{j}']['balanced_accuracy'] for j in range(int(len(result)/2))]
                grid_results += [(i, c, result)]
                balanced_accuracys_mean += [
                    np.mean(np.nan_to_num(balanced_accuracys))]
        candidates = list(zip(grid_results, balanced_accuracys_mean))
        candidates = sorted(candidates, key=lambda x: x[1], reverse=True)

        best_candidate_per_clf = []
        for clf in CLASSIFIER_MODES:
            for (i, c, cv), balanced_accuracy_mean in candidates:
                if c[3] == clf:
                    if cv['param'] is not None:
                        cv['param'] = {k.split('__')[-1]: v
                                       for k, v in cv['param'].items()}

                    best_candidate_per_clf += [((i, c, cv),
                                                balanced_accuracy_mean)]
                    break
        return best_candidate_per_clf

        # raise NotImplementedError
    else:
        raise ValueError(f"Unknown mode: {mode}")

In [38]:
path_input_model_selection_result = "./output/pval_filter_60_MVI/output_12to18_yesmental/results.pkl"
path_input_preprocessed_data_dir = "./output/pval_filter_60_MVI/output_12to18_yesmental/preprocessed"
path_input_data_raw = "./output/pval_filter_60_MVI/output_12to18_yesmental/data_cleaned_encoded.csv"
path_output_dir = "./output/pval_filter_60_MVI/output_12to18_yesmental/balanced_accuracy/test"
feature_label = "y12to18_Dep_YN_216m"
use_smote = True
use_smote_first = False
all_best_confusion_matrices_test = False
use_f1 = False
feature_kfold = None
random_state = 42

In [39]:
if not os.path.exists(path_output_dir):
        os.mkdir(path_output_dir)

model_selection_result = None
with open(path_input_model_selection_result, 'rb') as f:
    model_selection_result = pickle.load(f)
if use_f1:
    mode = "f1"
    best_candidate_per_clf = parse_model_selection_result(
        model_selection_result, mode)
else:
    mode = "balanced_accuracy"
    best_candidate_per_clf = parse_model_selection_result(
        model_selection_result, mode)

In [40]:
# best_candidate_per_clf

In [41]:
# use_f1 = True

In [42]:
# if not os.path.exists(path_output_dir):
#         os.mkdir(path_output_dir)

# model_selection_result = None
# with open(path_input_model_selection_result, 'rb') as f:
#     model_selection_result = pickle.load(f)
# if use_f1:
#     mode = "f1"
#     best_candidate_per_clf = parse_model_selection_result(
#         model_selection_result, mode)
# else:
#     mode = "balanced_accuracy"
#     best_candidate_per_clf = parse_model_selection_result(
#         model_selection_result, mode)

In [43]:
# best_candidate_per_clf

In [44]:
best_candidate = max(best_candidate_per_clf, key=lambda x: x[1])
_, best_combination, best_cv_result = best_candidate[0]
best_scale_mode, best_impute_mode, best_outlier_mode, best_clf \
    = best_combination
# print(best_combination)
pd.DataFrame(best_candidate_per_clf).to_csv(
    f"{path_output_dir}/best_clfs.csv")

In [45]:
X_train, y_train = load_X_and_y(
    f"{path_input_preprocessed_data_dir}/"
    f"{best_scale_mode}_{best_impute_mode}_{best_outlier_mode}_train.csv",
    col_y=feature_label)
X_test, y_test = load_X_and_y(
    f"{path_input_preprocessed_data_dir}/"
    f"{best_scale_mode}_{best_impute_mode}_{best_outlier_mode}_test.csv",
    col_y=feature_label)

In [46]:
splits = KFold_by_feature(
    X=X_train,
    y=y_train,
    n_splits=5,
    feature=feature_kfold,
    random_state=random_state)
if feature_kfold is not None:
    X_train = X_train.drop([feature_kfold], axis=1)
    X_test = X_test.drop([feature_kfold], axis=1)

clf = ClassifierHandler(
    classifier_mode=best_clf,
    params=best_cv_result['param'],
    random_state=ModelSelectionConfig.RNG_SMOTE).clf

# RFE/SFS
# Calculate and plot feature selection for the best model.
if use_f1:
    mode = "f1"
else:
    mode = "balanced_accuracy"

del best_cv_result['param']
plot_all_confusion_matrices(clf, X_train, y_train,
                            X_test, y_test, path_output_dir,
                            use_smote_first=False, use_rfe=False,
                            use_f1=use_f1, splits=splits)

/Users/arielleyoo/Desktop/Tagkopoulos_Lab/DepressionProj/MSAP/msap/utils/plot.py:407: RuntimeWarning: invalid value encountered in true_divide
  npvs = tns / (tns + fns)
/Users/arielleyoo/opt/anaconda3/envs/.depressionnew/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/arielleyoo/opt/anaconda3/envs/.depressionnew/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/arielleyoo/opt/anaconda3/envs/.depressionnew/lib/python3.10/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/arielleyoo/opt/anaconda3/envs/.depressionnew/lib/python3.10/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, di